# linux

[toc]

## 开机自启动
本文介绍通过 /etc/rc.local 文件配置开机自启动服务的配置步骤。
```
# show rc.local info
$ ls -ahl /etc/rc.local
lrwxrwxrwx. 1 root root 13 Jul 23 17:33 /etc/rc.local -rc.d/rc.local

# /etc/rc.local 是 /etc/rc.d/rc.local 软连接，检查 /etc/rc.d/rc.local 文件权限。
$ ls -ahl /etc/rc.d/rc.local
-rw-r--r--. 1 root root 514 Jul 27 10:31 /etc/rc.d/rc.local

# /etc/rc.d/rc.local 赋执行权限
$ chmod +x /etc/rc.d/rc.local 
$ ls -ahl /etc/rc.d/rc.local 
-rwxr-xr-x. 1 root root 514 Jul 27 10:31 /etc/rc.d/rc.local

# 添加开机自启动配置，在文件尾行追加服务启动命令
$ vi /etc/rc.local 

# 例如追加 ntpd 服务自启动
$ cat /etc/rc.local
#!/bin/bash
...
...
/usr/sbin/ntpd

# 重启服务器，检查服务是否可以开机后自启动
$ shutdown -r now
$ ps -ef | grep ntp
root        918      1  0 11:07 ?        00:00:00 /usr/sbin/ntpd
root       1434   1418  0 11:08 pts/0    00:00:00 grep --color=auto ntp
```

## 免密登录
集群主机列表：
192.168.153.121   host1.centos
192.168.153.122   host2.centos
192.168.153.123   host3.centos
192.168.153.124   host4.centos
其中主机 192.168.153.121   host1.centos 作为客户端主机，其他三台主机作为服务端主机，配置从 host1.centos 免密登录其他三台主机。

- 生成密钥
  ```
  # 任选一台主机生成密钥，此例选取 host1.centos 主机
  $ ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  # 命令执行后一路回车，最终生成密钥对存放到 ~/.ssh/ 路径下
  ```

- 客户端主机配置
  ```
  # 将私钥放到客户端主机 ~/.ssh/ 目录下，此处私钥已经在客户端主机 ~/.ssh/ 目录下
  # 在客户端主机执行下面命令，修改文件权限
  $ chmod 700 ~/.ssh
  $ chmod 600 ~/.ssh/id_rsa 
  ```

- 服务器端配置
  ```
  # 将公钥 id_rsa.pub 分别追加到服务端各主机 ~/.ssh/authorized_keys 文件中，以下操作均在客户端主机 host1.centos 上进行
  # 上传 id_rsa.pub 文件到服务端各主机临时文件 ~/.ssh/id_rsa.pub.tmp
  $ for i in {2..4}; do scp id_rsa.pub root@192.168.153.12$i:~/.ssh/id_rsa.pub.tmp; done

  # 将各服务端主机临时公钥文件 ~/.ssh/id_rsa.pub.tmp 追加到 ~/.ssh/authorized_keys 文件中
  $ for i in {2..4}; do ssh root@192.168.153.12$i "cat ~/.ssh/id_rsa.pub.tmp >~/.ssh/authorized_keys"; done

  # 删除服务端各主机上的临时公钥文件 ~/.ssh/id_rsa.pub.tmp 该步已实现免密访问
  $ for i in {2..4}; do ssh root@192.168.153.12$i "rm ~/.ssh/id_rsa.pub.tmp"; done
  ```

## 磁盘LVM扩容
以下介绍磁盘扩容的具体过程，大概过程分物理磁盘扩容前准备、磁盘使用情况查看、磁盘分区、文件系统扩容几个步骤。下面以 root 对应存储扩容为例.
- 物理磁盘扩容前准备
  该步完成物理主机将新的硬盘插入硬盘槽，或者虚拟机通过（主机 -虚拟机设置 -硬盘 -添加 or 扩展）等操作，以实现物理磁盘的扩容操作前的准备工作。

- 磁盘使用概况
  ```
  # 查看当前系统文件挂载目录磁盘使用情况
  $ df -h
  Filesystem               Size  Used Avail Use% Mounted on
  devtmpfs                 898M     0  898M   0% /dev
  tmpfs                    910M     0  910M   0% /dev/shm
  tmpfs                    910M  9.5M  901M   2% /run
  tmpfs                    910M     0  910M   0% /sys/fs/cgroup
  /dev/mapper/centos-root  8.0G  2.2G  5.8G  28% /
  /dev/sda1               1014M  151M  864M  15% /boot
  tmpfs                    182M     0  182M   0% /run/user/0

  # 查看当前磁盘分区情况
  $ fdisk -l

  Disk /dev/sda: 26.8 GB, 26843545600 bytes, 52428800 sectors
  Units = sectors of 1 * 512 = 512 bytes
  Sector size (logical/physical): 512 bytes / 512 bytes
  I/O size (minimum/optimal): 512 bytes / 512 bytes
  Disk label type: dos
  Disk identifier: 0x000bd23f

    Device Boot      Start         End      Blocks   Id  System
  /dev/sda1   *        2048     2099199     1048576   83  Linux
  /dev/sda2         2099200    20971519     9436160   8e  Linux LVM

  Disk /dev/mapper/centos-root: 8585 MB, 8585740288 bytes, 16769024 sectors
  Units = sectors of 1 * 512 = 512 bytes
  Sector size (logical/physical): 512 bytes / 512 bytes
  I/O size (minimum/optimal): 512 bytes / 512 bytes


  Disk /dev/mapper/centos-swap: 1073 MB, 1073741824 bytes, 2097152 sectors
  Units = sectors of 1 * 512 = 512 bytes
  Sector size (logical/physical): 512 bytes / 512 bytes
  I/O size (minimum/optimal): 512 bytes / 512 bytes
  ```
- 创建新分区
  通过 fdisk 或者 gdisk 命令创建新的分区。
  ```
  # 此通过 fdisk 命令创建分区，/dev/sda 为磁盘设备标识。
  $ fdisk /dev/sda
  Welcome to fdisk (util-linux 2.23.2).

  Changes will remain in memory only, until you decide to write them.
  Be careful before using the write command.

  # 输入 n 创建新的磁盘分区
  Command (m for help): n
  Partition type:
    p   primary (2 primary, 0 extended, 2 free)
    e   extended
  
  # 此处创建主分区，输入 p
  Select (default p): p

  # 输入分区号，使用默认值3（默认值3可以不输入直接按回车）
  Partition number (3,4, default 3): 3

  # 输入磁盘的起止始扇区编号，此处均使用默认扇区编号，代表将磁盘剩余空间全部分到该分区。若想使用默认值，也可以不用输入直接回车。
  # 输入起始扇区编号
  First sector (20971520-52428799, default 20971520): 20971520
  # 输入结束扇区编号
  Last sector, +sectors or +size{K,M,G} (20971520-52428799, default 52428799): 52428799
  Partition 3 of type Linux and of size 15 GiB is set

  # 修改新分区的分区类型（执行 fdisk /dev/sda 命令后输入 t 修改分区类型）
  Command (m for help): t
  # 选择要修改的分区编号，此处选择上面新建的3号分区
  Partition number (1-3, default 3): 3
  # 输入要修改为的分区类型所代表的编号（可输入 L 查看所有分区类型表），此处要修改为的分区类型为 'Linux LVM'
  Hex code (type L to list all codes): 8e
  Changed type of partition 'Linux' to 'Linux LVM'

  # 输入 w 保存以上操作配置，完成新建分区的操作
  Command (m for help): w
  The partition table has been altered!

  Calling ioctl() to re-read partition table.

  WARNING: Re-reading the partition table failed with error 16: Device or resource busy.
  The kernel still uses the old table. The new table will be used at
  the next reboot or after you run partprobe(8) or kpartx(8)
  Syncing disks.

  # 重启主机或者执行 partprobe 命令，刷新分区表. (该步骤可根据情况从 1,2 中任选一种方式实现)
  
  # 1.刷新分区表
  $ partprobe /dev/sda
  Warning: Unable to open /dev/sr0 read-write (Read-only file system).  /dev/sr0 has been opened read-only.

  # 2.partx 让内核重读分区表
  $ partx -a /dev/sda

  # 3.通过重启刷新分区表
  $ shutdown -r now
  ```
- 文件系统扩容
  给新建的分区建立文件系统,以供操作系统使用
  ```
  # 查看当前物理卷
  $ pvscan
    PV /dev/sda2   VG centos          lvm2 [<9.00 GiB / 0    free]
    Total: 1 [<9.00 GiB] / in use: 1 [<9.00 GiB] / in no VG: 0 [0   ]

  # 为新建的分区创建新的物理卷
  $ pvcreate /dev/sda3
  Physical volume "/dev/sda3" successfully created.

  # 查看卷分组,此处 root 对应 vg 为 centos.
  $ lvscan
    ACTIVE            '/dev/centos/swap' [1.00 GiB] inherit
    ACTIVE            '/dev/centos/root' [<8.00 GiB] inherit

  # 查看vg信息
  $ vgdisplay centos
  --- Volume group ---
  VG Name               centos
  System ID             
  Format                lvm2
  Metadata Areas        1
  Metadata Sequence No  3
  VG Access             read/write
  VG Status             resizable
  MAX LV                0
  Cur LV                2
  Open LV               2
  Max PV                0
  Cur PV                1
  Act PV                1
  VG Size               <9.00 GiB
  PE Size               4.00 MiB
  Total PE              2303
  Alloc PE / Size       2303 / <9.00 GiB
  Free  PE / Size       0 / 0   
  VG UUID               LmYLty-eRIE-sdhH-fItf-7pTg-TnrL-OUxOTp

  # vg扩容
  $ vgextend centos /dev/sda3
  Volume group "centos" successfully extended

  # 查看vg扩容后信息
  $ vgdisplay centos
  --- Volume group ---
  VG Name               centos
  System ID             
  Format                lvm2
  Metadata Areas        2
  Metadata Sequence No  4
  VG Access             read/write
  VG Status             resizable
  MAX LV                0
  Cur LV                2
  Open LV               2
  Max PV                0
  Cur PV                2
  Act PV                2
  VG Size               23.99 GiB
  PE Size               4.00 MiB
  Total PE              6142
  Alloc PE / Size       2303 / <9.00 GiB
  Free  PE / Size       3839 / <15.00 GiB
  VG UUID               LmYLty-eRIE-sdhH-fItf-7pTg-TnrL-OUxOTp

  # lv扩容,通过上一步可以看出可用可用PE为3839,所以下面最多可以分配参数值为3839
  $ lvextend -l +3839 /dev/centos/root 
  Size of logical volume centos/root changed from <8.00 GiB (2047 extents) to 22.99 GiB (5886 extents).
  Logical volume centos/root successfully resized.

  # 也可以分配指定lv大小的扩展容量
  $ lvextend -L +10G /dev/centos/root 
  Insufficient free space: 3840 extents needed, but only 3839 available

  # 文件系统扩容
  $ xfs_growfs /dev/centos/root
  meta-data=/dev/mapper/centos-root isize=512    agcount=4, agsize=524032 blks
          =                       sectsz=512   attr=2, projid32bit=1
          =                       crc=1        finobt=0 spinodes=0
  data     =                       bsize=4096   blocks=2096128, imaxpct=25
          =                       sunit=0      swidth=0 blks
  naming   =version 2              bsize=4096   ascii-ci=0 ftype=1
  log      =internal               bsize=4096   blocks=2560, version=2
          =                       sectsz=512   sunit=0 blks, lazy-count=1
  realtime =none                   extsz=4096   blocks=0, rtextents=0
  data blocks changed from 2096128 to 6027264

  # 查看扩容结果
  $ df -h
  Filesystem               Size  Used Avail Use% Mounted on
  devtmpfs                 898M     0  898M   0% /dev
  tmpfs                    910M     0  910M   0% /dev/shm
  tmpfs                    910M  9.6M  901M   2% /run
  tmpfs                    910M     0  910M   0% /sys/fs/cgroup
  /dev/mapper/centos-root   23G  2.2G   21G  10% /
  /dev/sda1               1014M  151M  864M  15% /boot
  tmpfs                    182M     0  182M   0% /run/user/0
  ```